In [30]:
import pandas as pd
from config import connect_databases

# Conectar a las bases de datos
db_op, db_etl = connect_databases()


In [ ]:
# Extraer los datos de la tabla mensajeria_servicio
query = "SELECT fecha_solicitud FROM public.mensajeria_servicio WHERE activo = true"
servicios = pd.read_sql_query(query, db_op)

# Convertir la columna 'fecha_solicitud' a datetime
servicios['fecha_solicitud'] = pd.to_datetime(servicios['fecha_solicitud'], errors='coerce')

# Extraer el mes de la fecha
servicios['mes'] = servicios['fecha_solicitud'].dt.month_name()

# Calcular la cantidad de servicios por mes
servicios_por_mes = servicios.groupby('mes').size().reset_index(name='numero_servicios')

# Mostrar el resultado
print(servicios_por_mes)


          mes  numero_servicios
0       April              4180
1      August              4184
2    December                23
3    February              2232
4     January               272
5        July              4399
6        June              3998
7       March              3130
8         May              4506
9    November                 6
10    October                 5
11  September                17


In [32]:
# Extraer el día de la fecha_solicitud
servicios['dia'] = servicios['fecha_solicitud'].dt.day_name()

# Calcular la cantidad de servicios por día
servicios_por_dia = servicios.groupby('dia').size().reset_index(name='numero_servicios')

# Mostrar el resultado
print(servicios_por_dia)


         dia  numero_servicios
0     Friday              5010
1     Monday              4107
2   Saturday              2327
3     Sunday               793
4   Thursday              4902
5    Tuesday              5126
6  Wednesday              4687


In [34]:
import pandas as pd
from config import connect_databases

# Extraer los datos de los estados de los servicios
query = """
    SELECT ms.id AS key_servicio, es_inicio.hora AS hora_iniciado
    FROM mensajeria_servicio ms
    LEFT JOIN mensajeria_estadosservicio es_inicio 
    ON ms.id = es_inicio.servicio_id AND es_inicio.estado_id = 1
    WHERE ms.activo = true
"""
db_op, db_etl = connect_databases()
inicios = pd.read_sql_query(query, db_op)

# Verificar si hay valores nulos en 'hora_iniciado'
print(inicios['hora_iniciado'].isnull().sum())

# Reemplazar los valores nulos con un valor de hora predeterminado (por ejemplo, 00:00)
inicios['hora_iniciado'] = inicios['hora_iniciado'].fillna('00:00:00')

# Convertir la columna 'hora_iniciado' a tipo timedelta
inicios['hora_iniciado'] = pd.to_timedelta(inicios['hora_iniciado'].astype(str), errors='coerce')

# Extraer la hora (0-23)
inicios['hora'] = inicios['hora_iniciado'].dt.seconds // 3600

# Calcular la cantidad de servicios por hora
servicios_por_hora = inicios.groupby('hora').size().reset_index(name='numero_servicios')

# Mostrar el resultado
print(servicios_por_hora)


1
    hora  numero_servicios
0      0               137
1      1               156
2      2               147
3      3               102
4      4               100
5      5               130
6      6               876
7      7              1050
8      8              2614
9      9              3367
10    10              2925
11    11              3270
12    12              1743
13    13              1435
14    14              2527
15    15              2619
16    16              2100
17    17              1354
18    18               423
19    19               332
20    20               214
21    21               165
22    22               170
23    23               152


In [35]:
# Extraer datos de la tabla clientes_usuarioaquitoy
query = """
    SELECT cu.cliente_id, ms.fecha_solicitud
    FROM mensajeria_servicio ms
    JOIN clientes_usuarioaquitoy cu ON ms.usuario_id = cu.id
    WHERE ms.activo = true
"""
servicios_cliente = pd.read_sql_query(query, db_op)

# Convertir la columna 'fecha_solicitud' a datetime
servicios_cliente['fecha_solicitud'] = pd.to_datetime(servicios_cliente['fecha_solicitud'], errors='coerce')

# Extraer el mes de la fecha
servicios_cliente['mes'] = servicios_cliente['fecha_solicitud'].dt.month_name()

# Calcular la cantidad de servicios por cliente y mes
servicios_por_cliente_mes = servicios_cliente.groupby(['cliente_id', 'mes']).size().reset_index(name='numero_servicios')

# Mostrar el resultado
print(servicios_por_cliente_mes)


     cliente_id       mes  numero_servicios
0             2     April                 2
1             2    August                 1
2             2  December                17
3             2  February                 9
4             2   January                22
..          ...       ...               ...
103          25      July               170
104          25      June               148
105          25     March               115
106          25       May               150
107          27    August                 1

[108 rows x 3 columns]
